# Walmart Stocks Data Analysis Using Pyspark


Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the 10 years. 

Here is what walmart says about there latest data:
"Our historical data has the Open, High, Low, Close, and volume numbers along with Walmart P/E ratio, and PS ratio. The Walmart stock price history chart shows that the stock price reached a high of 90.47 on 08 Jan, 2015, and a low of 42.27 on 08 Jan, 2015 in last five years."

# Using the walmart_stock.csv file for Analysis

## Starting a simple Spark Session

In [18]:
from pyspark.sql import SparkSession 
spark =   SparkSession.builder.appName("Walmart Stock").getOrCreate()



####  Walmart stock

In [19]:
walmart_df = spark.read.csv("WalmartStock_2017.csv",header=True, inferSchema=True)

#### Features or columns

In [21]:
walmart_df.columns

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Price to Earnings Ratio',
 'Price to Sales Ratio']

#### What does the Schema look like?

In [22]:
walmart_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Price to Earnings Ratio: string (nullable = true)
 |-- Price to Sales Ratio: string (nullable = true)



#### Features Observations

In [23]:
walmart_df.take(5)

[Row(Date=datetime.datetime(2017, 9, 12, 0, 0), Open=79.3, High=79.91, Low=79.17, Close=79.61, Volume=6004802, Price to Earnings Ratio='19.1370', Price to Sales Ratio='0.4887'),
 Row(Date=datetime.datetime(2017, 9, 11, 0, 0), Open=79.15, High=79.35, Low=78.66, Close=79.08, Volume=7000202, Price to Earnings Ratio='19.0100', Price to Sales Ratio='0.4854'),
 Row(Date=datetime.datetime(2017, 9, 8, 0, 0), Open=79.72, High=79.875, Low=77.73, Close=78.88, Volume=9999593, Price to Earnings Ratio='18.9620', Price to Sales Ratio='0.4842'),
 Row(Date=datetime.datetime(2017, 9, 7, 0, 0), Open=80.32, High=80.76, Low=80.0, Close=80.12, Volume=6820634, Price to Earnings Ratio='19.2600', Price to Sales Ratio='0.4918'),
 Row(Date=datetime.datetime(2017, 9, 6, 0, 0), Open=79.88, High=80.4, Low=79.72, Close=80.08, Volume=8425487, Price to Earnings Ratio='19.2500', Price to Sales Ratio='0.4916')]

#### Use describe() to learn about the DataFrame.

In [6]:
walmart_df.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+-----------------------+--------------------+
|summary|              Open|              High|               Low|             Close|              Volume|Price to Earnings Ratio|Price to Sales Ratio|
+-------+------------------+------------------+------------------+------------------+--------------------+-----------------------+--------------------+
|  count|              2665|              2665|              2665|              2665|                2665|                   2665|                2665|
|   mean|61.456234521575894|61.933444277673615| 61.01721313320817|63.655750469043035|1.1900423796247656E7|      14.93059052059055|  0.4961338772338774|
| stddev|16.135985131238954|16.196346452770083|16.086215691575212| 11.46608356312984|   7962357.992945932|      1.641863172019808| 0.04618966587390132|
|    min|               0.0|               0.0|               0.0|             42.27|   

In [7]:
walmart_df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Price to Earnings Ratio: string (nullable = true)
 |-- Price to Sales Ratio: string (nullable = true)



In [24]:
# for better understanding 
# see columns below
result = walmart_df.describe()
result

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Price to Earnings Ratio: string, Price to Sales Ratio: string]

In [25]:
from pyspark.sql.functions import format_number
result.select(result['summary'], result['Volume'].cast('int').alias('Volume'), format_number(result['Open'].cast('float'),2).alias('Open')
             ,format_number(result['High'].cast('float'),2).alias('High'),
             format_number(result['Low'].cast('float'),2).alias('Low'),
             format_number(result['Close'].cast('float'),2).alias('Close'),
              format_number(result['Price to Earnings Ratio'].cast('float'),2).alias('Price to Earnings Ratio'),
              format_number(result['Price to Sales Ratio'].cast('float'),2).alias('Price to Sales Ratio')
             ).show()

+-------+--------+--------+--------+--------+--------+-----------------------+--------------------+
|summary|  Volume|    Open|    High|     Low|   Close|Price to Earnings Ratio|Price to Sales Ratio|
+-------+--------+--------+--------+--------+--------+-----------------------+--------------------+
|  count|    2665|2,665.00|2,665.00|2,665.00|2,665.00|               2,665.00|            2,665.00|
|   mean|    null|   61.46|   61.93|   61.02|   63.66|                  14.93|                0.50|
| stddev| 7962357|   16.14|   16.20|   16.09|   11.47|                   1.64|                0.05|
|    min|       0|    0.00|    0.00|    0.00|   42.27|                  10.55|                0.37|
|    max|92820440|   90.80|   90.97|   89.25|   90.47|                   null|                null|
+-------+--------+--------+--------+--------+--------+-----------------------+--------------------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [26]:
walmart_df2 = walmart_df.withColumn('Ratio HV',walmart_df['High'] /walmart_df['Volume'])

walmart_df2.show()

+-------------------+-----+------+-----+-----+--------+-----------------------+--------------------+--------------------+
|               Date| Open|  High|  Low|Close|  Volume|Price to Earnings Ratio|Price to Sales Ratio|            Ratio HV|
+-------------------+-----+------+-----+-----+--------+-----------------------+--------------------+--------------------+
|2017-09-12 00:00:00| 79.3| 79.91|79.17|79.61| 6004802|                19.1370|              0.4887|1.330768275123809...|
|2017-09-11 00:00:00|79.15| 79.35|78.66|79.08| 7000202|                19.0100|              0.4854|1.133538717882712...|
|2017-09-08 00:00:00|79.72|79.875|77.73|78.88| 9999593|                18.9620|              0.4842|7.987825104481753E-6|
|2017-09-07 00:00:00|80.32| 80.76| 80.0|80.12| 6820634|                19.2600|              0.4918|1.184054150977753...|
|2017-09-06 00:00:00|79.88|  80.4|79.72|80.08| 8425487|                19.2500|              0.4916|9.542475111527678E-6|
|2017-09-05 00:00:00|77.

#### What day had the Peak High in Price?

In [27]:
walmart_df.orderBy(walmart_df['High'].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

#### What is the mean of the Close column?

In [12]:
from pyspark.sql.functions import mean
walmart_df.select(mean(walmart_df["Close"])).show()

+------------------+
|        avg(Close)|
+------------------+
|63.655750469043035|
+------------------+



#### What is the max and min of the Volume column?

In [46]:
from pyspark.sql.functions import mean, max, min
walmart_df.select(mean(walmart_df["Volume"])).show()

+--------------------+
|         avg(Volume)|
+--------------------+
|1.1900423796247656E7|
+--------------------+



In [47]:
#walmart_df.show()
walmart_df.select(max(walmart_df["Volume"]),max(walmart_df["Volume"])).show()


+-----------+-----------+
|max(Volume)|max(Volume)|
+-----------+-----------+
|   92820440|   92820440|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [48]:
walmart_df.filter("Close < 60").show()

+-------------------+-----+------+------+-----+--------+-----------------------+--------------------+
|               Date| Open|  High|   Low|Close|  Volume|Price to Earnings Ratio|Price to Sales Ratio|
+-------------------+-----+------+------+-----+--------+-----------------------+--------------------+
|2015-12-21 00:00:00|58.82| 59.58| 58.67|59.55| 9645451|                12.7520|              0.3949|
|2015-12-18 00:00:00|58.92|  59.4| 58.75|58.85|16256725|                12.6020|              0.3903|
|2015-12-17 00:00:00|60.33| 60.33| 58.97|58.98|12976095|                12.6300|              0.3911|
|2015-12-15 00:00:00|60.64| 60.84| 59.57|59.64|14615439|                12.7710|              0.3955|
|2015-12-11 00:00:00|59.05| 59.53| 58.81|59.36| 9744977|                12.7110|              0.3937|
|2015-12-10 00:00:00|59.13| 60.09| 59.09|59.56|11359446|                12.7540|              0.3950|
|2015-12-09 00:00:00|59.26| 60.11|  58.9|59.13| 8981982|                12.6620|  

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [49]:
( walmart_df.filter('High > 80').count()/ walmart_df.count())*100

5.215759849906192

#### What is the Pearson correlation between High and Volume?


In [58]:
from pyspark.sql.functions import *
walmart_df.select(corr("High","Volume")).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.18667006516302553|
+--------------------+



#### What is the max High per year?

In [62]:
yeardf = walmart_df.withColumn("Year",year(walmart_df["Date"]))
#yeardf.show()
max_df = yeardf.select('High','Close','Date','Year').groupBy('Year').agg(max('High'))
max_df.show()
#Or
#max_df.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2007|    50.05|
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|     77.6|
|2009|    57.51|
|2016|    75.19|
|2010|    56.27|
|2011|     60.0|
|2008|    63.85|
|2017|    81.99|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [60]:
monthdf = walmart_df.withColumn('month',month('Date'))
monthdf.show()

monthavgs = monthdf.select("Month","Close").groupBy("Month").mean()
monthavgs.select("Month",'avg(Close)').orderBy('Month').show()

+-------------------+-----+------+-----+-----+--------+-----------------------+--------------------+-----+
|               Date| Open|  High|  Low|Close|  Volume|Price to Earnings Ratio|Price to Sales Ratio|month|
+-------------------+-----+------+-----+-----+--------+-----------------------+--------------------+-----+
|2017-09-12 00:00:00| 79.3| 79.91|79.17|79.61| 6004802|                19.1370|              0.4887|    9|
|2017-09-11 00:00:00|79.15| 79.35|78.66|79.08| 7000202|                19.0100|              0.4854|    9|
|2017-09-08 00:00:00|79.72|79.875|77.73|78.88| 9999593|                18.9620|              0.4842|    9|
|2017-09-07 00:00:00|80.32| 80.76| 80.0|80.12| 6820634|                19.2600|              0.4918|    9|
|2017-09-06 00:00:00|79.88|  80.4|79.72|80.08| 8425487|                19.2500|              0.4916|    9|
|2017-09-05 00:00:00|77.95| 79.89| 77.9| 79.8|12068490|                19.1830|              0.4899|    9|
|2017-09-01 00:00:00|78.31|  78.5|78.